In [ ]:
%load_ext autoreload
%autoreload 2
from utils import process_subject, run_openpose, json2np, peakdet, get_angle, VERT, MHIP, NECK
import os
import numpy as np
import pandas as pd
import pickle
import subprocess
import glob

import csv
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure
from scipy.signal import savgol_filter
from utils import smooth_ts

from scipy.stats import linregress

In [ ]:
# Get all videos for lab validation
video_paths = glob.glob("data/lab/*/*.mov")

def get_slug(video_path):
    return video_path[9:].replace("/","-").replace(" ","-").replace("(","").replace(")","")
for path in video_paths:
    print(path)

In [ ]:
# Run openpose on all videos
for path in video_paths:
    slug = get_slug(path)[:-4]
    run_openpose(path, slug)

In [ ]:
# Convert keypoint jsons to numpy arrays
os.makedirs("data/lab/arrays/", exist_ok=True)
for subjectid in glob.glob("data/lab/keypoints/*"):
    target_path = "data/lab/arrays/{}.npy".format(subjectid.split("/")[-1])
    if os.path.isfile(target_path):
        print("{} exists".format(target_path))
        continue
    res = json2np(subjectid,"")
    np.save(target_path, res)

In [ ]:
# Compute metrics on mocap files and compare with openpose videos
def mocap_op_comp(path_mot):
    rows = []

    print(path_mot)

    with open(path_mot,"r") as f:
        lines = f.readlines()
        for k, row in enumerate(lines):
            if row.strip() == "endheader":
                break

        header = list(map(lambda x: x.strip(), lines[k+1].split("\t")))
        for row in lines[(k+2):]:
            variables = list(map(lambda x: float(x.strip()), row.split("\t")))
            rows.append(dict(zip(header,variables)))

    df = pd.DataFrame(rows)
    print(header)
    slug = get_slug(path_mot)[:-4]

    res = {"path_mot": path_mot}
    
    framerates = {
        "data/lab/subject 2/STSAsym1 (1).mot": 239.67,
        "data/lab/subject 2/STS1 (1).mot": 239.67,
        "data/lab/subject 3/STS1 (2).mot": 239.67,
        "data/lab/subject 3/STSweakLegs1 (2).mot": 239.67, 
        "data/lab/subject 3/STSAsym1 (2).mot": 239.67, 
        
    }
    framerate = int(framerates.get(path_mot, 60))

    try:
        results = process_subject(slug,
                    framerate = framerates.get(path_mot, 60),
                    processed_npy_path="data/lab/arrays/")
    except:
        return res

    oddify = lambda x: x if x%2 ==1 else x+1
    
    opose = np.load("data/lab/arrays/{}.npy".format(slug))

    pelvis_mc = df["pelvis_ty"]
    pelvis_mc = savgol_filter(pelvis_mc, oddify(framerate//2), 3)
    pelvis_mc = (pelvis_mc - pelvis_mc.min()) / (pelvis_mc.max() - pelvis_mc.min())

    filtered = savgol_filter(pelvis_mc, 3, 1)

    hip_op = -opose[:,3*8+1]
    hip_op = savgol_filter(hip_op, oddify(framerate//2), 3)
    hip_op = (hip_op - hip_op.min()) / (hip_op.max() - hip_op.min())

    SHIFT = 0 # -0.48 # <- to calculate automatically

    plt.plot(df["time"], filtered, label="Mocap")
    plt.plot([SHIFT+x/framerate for x in range(opose.shape[0])], savgol_filter(hip_op, oddify(framerate//2), 3), label="OpenPose")
    plt.legend()
    plt.show()
    
    vert = opose[:,(MHIP*3):(MHIP*3+3)].copy()
    vert[:,1] = vert[:,1] - 10
    opose = np.hstack([opose.copy(), vert])

    framerate_mc = ((df["time"] > 1) * (df["time"] <= 2) ).sum()

    trunk_lean_op = savgol_filter(results["trunk_lean_ts"] *180/np.pi, oddify(framerate//2), 3) - 180
    trunk_lean_mc = savgol_filter(-df["pelvis_tilt"].to_numpy(), oddify(framerate_mc//2), 3)
    start_frame = 160
    
    peaks_trunk_lean_op = peakdet(trunk_lean_op[20:-50], 10)
    peaks_trunk_lean_mc = peakdet(trunk_lean_mc[20:-20], 10)

    derv = lambda x: x[1:] - x[:-1]
    peaks = peakdet(pelvis_mc, 0.5)
    
    grid = np.arange(trunk_lean_op.shape[0])/framerate

    plt.plot(df["time"], trunk_lean_mc, label="Moocap")
    plt.plot(grid[:(-start_frame)], trunk_lean_op[start_frame:], label="OpenPose")
    plt.legend()
    plt.show()
    
    trunk_lean_vel_op = derv(trunk_lean_op) * framerate
    trunk_lean_vel_mc = derv(trunk_lean_mc) * framerate_mc

    plt.plot(df["time"][:-1], trunk_lean_vel_mc, label="Mocap")
    plt.plot(grid[:(-start_frame-1)], trunk_lean_vel_op[start_frame:], label="OpenPose")
    plt.legend()
    plt.show()

    trunk_lean_acc_op = derv(trunk_lean_vel_op) * framerate
    trunk_lean_acc_mc = derv(trunk_lean_vel_mc) * framerate_mc
    
    peak_trunk_lean_acc_op = np.quantile(trunk_lean_acc_op[20:-50], 0.95)
    peak_trunk_lean_acc_mc = np.quantile(trunk_lean_acc_mc[20:-20], 0.95)

    plt.plot(grid[:(-start_frame-2)], trunk_lean_acc_op[start_frame:], label="OpenPose")
    plt.plot(df["time"][:-2], trunk_lean_acc_mc, label="Moocap")
    plt.legend()
    plt.show()

    
    times_mocap = np.array(df["time"][derv(peaks[0][:,0])])
    peaks = peakdet(hip_op, 0.5)
    times_openpose = derv(peaks[0][:,0])/framerate
    
    if len(times_openpose) < 4 or len(times_mocap) < 4:
        res.update({
            "mean_diff": None,
        })
    else:
        res.update({
            "mean_diff": (times_openpose[:4] - times_mocap[:4]).mean(),
            "time_op": (times_openpose[:4]).mean(),
            "time_mc": (times_mocap[:4]).mean(),
        })
        
    print(sorted(peaks_trunk_lean_op[0][:,1])[-2])
    print(sorted(peaks_trunk_lean_mc[0][:,1])[-2])
    

    res.update({
        "trunk_lean_max_mean_op": sorted(peaks_trunk_lean_op[0][:,1])[-2],
        "trunk_lean_max_mean_mc": sorted(peaks_trunk_lean_mc[0][:,1])[-2],
        "trunk_lean_op": peaks_trunk_lean_op[0][:,1],
        "trunk_lean_mc": peaks_trunk_lean_mc[0][:,1],
        "trunk_lean_acc_op": peak_trunk_lean_acc_op,
        "trunk_lean_acc_mc": peak_trunk_lean_acc_mc,
    })
        
    return res

In [ ]:
# Test on one mocap file
mocap_op_comp("data/lab/subject 2/STSAsym1 (1).mot")

In [ ]:
# Run on all mocap files 
video_paths = glob.glob("data/lab/*/*.mot")

res = []
for video in video_paths:
    res.append(mocap_op_comp(video))

In [ ]:
# Print out results
res

In [ ]:
# Generate plots for the paper
res_df = pd.DataFrame(res).dropna()

plt.scatter(res_df["time_op"], res_df["time_mc"])
plt.show()

slope, intercept, r_value, p_value, std_err = linregress(res_df["time_op"], res_df["time_mc"])
print(r_value)

plt.scatter(res_df["trunk_lean_max_mean_op"], res_df["trunk_lean_max_mean_mc"])
plt.show()

slope, intercept, r_value, p_value, std_err = linregress(res_df["trunk_lean_max_mean_op"], res_df["trunk_lean_max_mean_mc"])
print(r_value)

plt.scatter(res_df["trunk_lean_acc_mc"], res_df["trunk_lean_acc_op"])
plt.show()

slope, intercept, r_value, p_value, std_err = linregress(res_df["trunk_lean_acc_op"], res_df["trunk_lean_acc_mc"])
print(r_value)